## Language Models

- Machine Translation
- Spell Correction
- Speech Recognition
- Summarisation, Q & A


### Formulations

- How likely a sentence is well formed
$$ P(S) = P(w_1,w_2,w_3,.....,w_n) $$

- Predict the next word - Mobile Keyboards
    $$ P(w_5 |  w_1,w_2,w_3,w_4)  $$

## The Hangman Game

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game, and a method allowing interactive play. 

Predict one character at a time, for a word to be predicted

- Predict one character at a time - Mobile Keyboards
    $$ P(c_5 |  c_1,c_2,c_3,c_4)  $$

The description and the methods **`hangman`** and **`human`** is obtained from University of Melbourne, from the course [Web Search and Text Analysis](http://people.eng.unimelb.edu.au/tcohn/comp90042.html)


In [1]:
# allowing better python 2 & python 3 compatibility 
from __future__ import print_function 


# max_mistakes = the number of mistakes one is allowed to make before guessing the actual word
# verbose = Flag to whether print the statements or not.
# guesser = function to be called. For example when we pass human that means wherever guesser() is written it will invoke human()
# secret_word = the word to be guessed.

def hangman(secret_word, guesser, max_mistakes=8, verbose=True):
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            try:
                guessed.add(guess)
            except:
                print(guessed,guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

def human(mask, guessed):
    print('Enter your guess:')
    return raw_input().lower().strip()
    #return input().lower().strip() # swap with above for python 3


## Loading the dataset

In [2]:
# Loading up the vocabulory
f = open('train.txt').read().splitlines()

In [3]:
test = set()
test_words = []
with open("test.txt", mode="r") as myFile:
    for line in myFile:
        try:
            test_words.append(line.strip().encode("utf-8"))
        except:
            pass
test = set(test_words)
train = set(f) - test

## Randomly predicting a character at a time

 - Randomly predict a number from 1 to 26. 
 - Pick the corresponding letter from the alphabet.
 - Check if its already guessed, else not perdict it

In [4]:
"""
    We can predict that any problem is tough based on the results of the algorithm if it is done randomly
"""

import numpy as np
import string    


def randomG(mask,guessed):
    
    # Initialise a dictionary with all the charcters in the alphbet
    d = dict.fromkeys(string.ascii_lowercase, 0)
    
    # Give an integer index for each character
    pd = dict()
    for i,j in enumerate(d.keys()):
        pd[i+1] = j
    
    while 1:
        trials = np.random.choice(range(1,27), 1,replace=False)
        if trials[0] not in guessed:
            return pd[trials[0]]

In [5]:
mist = hangman('whatever',randomG,8)

Starting hangman game. Target is _ _ _ _ _ _ _ _ length 8
You have 8 attempts remaining.
Guess is a
Good guess: _ _ a _ _ _ _ _
You have 8 attempts remaining.
Guess is k
Sorry, try again.
You have 7 attempts remaining.
Guess is r
Good guess: _ _ a _ _ _ _ r
You have 7 attempts remaining.
Guess is i
Sorry, try again.
You have 6 attempts remaining.
Guess is r
Already guessed this before.
You have 5 attempts remaining.
Guess is z
Sorry, try again.
You have 4 attempts remaining.
Guess is t
Good guess: _ _ a t _ _ _ r
You have 4 attempts remaining.
Guess is f
Sorry, try again.
You have 3 attempts remaining.
Guess is k
Already guessed this before.
You have 2 attempts remaining.
Guess is v
Good guess: _ _ a t _ v _ r
You have 2 attempts remaining.
Guess is a
Already guessed this before.
You have 1 attempts remaining.
Guess is a
Already guessed this before.
Out of guesses. The word was whatever


In [6]:
# Number of mistakes made in count of characters - Highest possible for 1000 words?
totMist = 0

# Number of words correctly predicted?
corrs = 0

#Testing for effectiveness of the method on the test data
for i,item in enumerate(test):
    mist = hangman(item, randomG, 8, False)
    totMist += mist
    if mist < 8:
        corrs += 1
        print(i,item)

# trying to predict for all the words in the train set. Brute Force Random .
print(totMist,corrs)

131 lemurine
2726 agrapha
4170 anaphalantiasis
8242 casern
8468 bodement
8918 zoonomy
9105 euphobia
10736 putcher
88924 8


## Can we do any better? ...In a principled and efficient way?

1) Chain Rule
$$ P(w_1,w_2,w_3,.....,w_n) \approx  P(w_1)\cdot P(w_2|w_1) \cdot P(w_3|w_2,w_1) ..... \cdot P(w_n | w_1,.....,w_{n-1}) $$

2) Markov Assumption


$$ P(w_1,w_2,w_3,.....,w_n) \approx \prod_{i} P(w_i | w_{i-k} ...w_{i-1} ) $$

$$ P(w_i |w_1w_2.....w_{i-1}) \approx  P(w_i | w_{i-k} ...w_{i-1} ) $$


## Unigram character prediction

- Predict a character based on the frequency of occurence in the corpus
$$ P(w_1,w_2,w_3,.....,w_n) = \prod_{i} P(w_i) $$

In [7]:
# Calculates the freuency of each character for all words combined
charaFreq = dict()
totFreq = 0
for item in train:
    for chara in item:
        try:
            charaFreq[chara] += 1
        except:
            charaFreq[chara] = 1            
        totFreq += 1
import operator
sorted_x = sorted(charaFreq.items(), key=operator.itemgetter(1),reverse=True)

sorted_x[:5]

[('a', 596440), ('e', 549015), ('i', 431272), ('r', 393747), ('n', 384726)]

In [8]:
def unig(mask,guessed):
    for item in sorted_x:
        if item[0] not in guessed:
            return item[0]
hangman('whatever', unig, 8, True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ length 8
You have 8 attempts remaining.
Guess is a
Good guess: _ _ a _ _ _ _ _
You have 8 attempts remaining.
Guess is e
Good guess: _ _ a _ e _ e _
You have 8 attempts remaining.
Guess is i
Sorry, try again.
You have 7 attempts remaining.
Guess is r
Good guess: _ _ a _ e _ e r
You have 7 attempts remaining.
Guess is n
Sorry, try again.
You have 6 attempts remaining.
Guess is s
Sorry, try again.
You have 5 attempts remaining.
Guess is o
Sorry, try again.
You have 4 attempts remaining.
Guess is t
Good guess: _ _ a t e _ e r
You have 4 attempts remaining.
Guess is l
Sorry, try again.
You have 3 attempts remaining.
Guess is c
Sorry, try again.
You have 2 attempts remaining.
Guess is m
Sorry, try again.
You have 1 attempts remaining.
Guess is u
Sorry, try again.
Out of guesses. The word was whatever


8

In [9]:
totMist = 0
corrs = 0
for i,item in enumerate(test):
    mist = hangman(item, unig, 8, False)
    totMist += mist
    if mist < 8:
        corrs += 1

corrs,totMist

(2538, 81427)

## Unigram with Length

In [10]:
from collections import defaultdict
frequni = defaultdict(dict)
for item in train:
    for chara in item:
        try:
            frequni[len(item)][chara] += 1
        except:
            frequni[len(item)][chara] = 1

In [11]:
def unigLen(mask,guessed):
    sorted_x = sorted(frequni[len(mask)].items(), key=operator.itemgetter(1),reverse=True)
    for item in sorted_x:
        if item[0] not in guessed:
            return item[0]

# Exploiting the fact that the words with different lengths have different alphabet distributions
        
totMist = 0
corrs = 0
for i,item in enumerate(test):
    mist = hangman(item, unigLen, 8, False)
    totMist += mist
    if mist < 8:
        corrs += 1
print (corrs,totMist)

TypeError: 'in <string>' requires string as left operand, not NoneType

## Unigram with Position

- Probability distribution conditioned on individual position


In [12]:
from collections import defaultdict
frequni = defaultdict(dict)
posCountDict = dict()
for item in train:
    for pos,chara in enumerate(item):
        try:
            frequni[pos][chara] += 1
        except KeyError:
            frequni[pos][chara] = 1
        try:
            posCountDict[pos] += 1
        except KeyError:
            posCountDict[pos] = 1

# Exploiting the fact that each alphabet has a different distrbution based on what position it occurs

In [14]:
# We have to calculate the probabilities because the denominators are no longer same. Thus to avoid positional bias we calculate probabilities
freqProb = defaultdict(dict)
for item in frequni.keys():
    if posCountDict[item] > 50:
        for stuff in frequni[item].keys():
            freqProb[item][stuff] = frequni[item][stuff]*1.0/posCountDict[item]

In [15]:
def unigPos(mask,guessed):
    locDict = dict()
    for item in freqProb.keys():
        if item <= len(mask):
            sorted_x = sorted(freqProb[item].items(), key=operator.itemgetter(1),reverse=True)
            for stuff in sorted_x:
                if stuff[0] not in guessed:
                    if stuff[0] in locDict.keys():
                        if stuff[1] > locDict[stuff[0]]:
                            locDict[stuff[0]] = stuff[1]
                            break
                    else:
                        locDict[stuff[0]] = stuff[1]
                        break
    sorted_x = sorted(locDict.items(), key=operator.itemgetter(1),reverse=True)
    for stuff in sorted_x:
        if stuff[0] not in guessed:
            return stuff[0]
    
    print('onnumilla')
        
totMist = 0
corrs = 0
for i,item in enumerate(test):
    mist = hangman(item, unigPos, 8, False)
    totMist += mist
    if mist < 8:
        corrs += 1
print (corrs,totMist)

2796 81870


- Shall we keep the max per position?

This is done beacuse we find all position toppers and then find the topper among them. Thus we have to ensure that only the updated value should be kept if the alphabet is the topper in two positions

```for stuff in sorted_x:
                if stuff[0] not in guessed:
                    if stuff[0] in locDict.keys():
                        if stuff[1] > locDict[stuff[0]]:
                            locDict[stuff[0]] = stuff[1]
                            break
                    else:
                        locDict[stuff[0]] = stuff[1]
                        break
```                        
                 
- Shall we add up the probabilities?

## Unigram with Position bidirectionally

In [16]:
from collections import defaultdict
frequni = defaultdict(dict)
posCountDict = dict()
frequniRev = defaultdict(dict)
posCountDictRev = dict()

# Trying to combine the probability distribution by traversing the word from left to right and right to left.
    
for item in train:
    for pos,chara in enumerate(item):
        revIndex = len(item)-1-pos

        try:
            frequni[pos][chara] += 1
        except KeyError:
            frequni[pos][chara] = 1
            
        try:
            frequniRev[revIndex][chara] += 1
        except KeyError:
            frequniRev[revIndex][chara] = 1
            
            
        try:
            posCountDict[pos] += 1
        except KeyError:
            posCountDict[pos] = 1
            
        try:
            posCountDictRev[revIndex] += 1
        except KeyError:
            posCountDictRev[revIndex] = 1


freqProb = defaultdict(dict)
freqProbRev = defaultdict(dict)

for item in frequni.keys():
    if posCountDict[item] > 50:
        for stuff in frequni[item].keys():
            freqProb[item][stuff] = frequni[item][stuff]*1.0/posCountDict[item]
            
for item in frequniRev.keys():
    if posCountDictRev[item] > 50:
        for stuff in frequniRev[item].keys():
            freqProbRev[item][stuff] = frequniRev[item][stuff]*1.0/posCountDictRev[item]

freqProb.keys(),freqProbRev.keys()

([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21])

In [17]:
def unigPosBi(mask,guessed):
    locDict = list()
    
    for item in freqProb.keys():
        if item <= len(mask):
            sorted_x = sorted(freqProb[item].items(), key=operator.itemgetter(1),reverse=True)
            for stuff in sorted_x:
                if stuff[0] not in guessed:
                    locDict.append((stuff[0],stuff[1],item))
                    break
    
    for item in freqProbRev.keys():
        if item <= len(mask):
            sorted_x = sorted(freqProbRev[item].items(), key=operator.itemgetter(1),reverse=True)
            for stuff in sorted_x:
                if stuff[0] not in guessed:
                    locDict.append((stuff[0],stuff[1],-1*item))
                    break
    loctDict2 = dict()
    
    for item in locDict:
        try:
            loctDict2[item[0]] += item[1]
        except:
            loctDict2[item[0]] = item[1]
    
    sorted_x = sorted(loctDict2.items(), key=operator.itemgetter(1),reverse=True)
    for stuff in sorted_x:
        if stuff[0] not in guessed:
            return stuff[0]
    
    print('onnumilla')
        
totMist = 0
corrs = 0
for i,item in enumerate(test):
    mist = hangman(item, unigPosBi, 8, False)
    totMist += mist
    if mist < 8:
        corrs += 1

print (corrs,totMist)

3093 80298


### bi-gram Models

$$
P(w_i | w_{i-1}) = \frac{count(w_{i-1},w_i)}{count(w_{i-1})} $$

In [18]:
from collections import defaultdict
bigram = defaultdict(dict)

for item in train:
    item2 = '$'+item+'$'
    for i,chara in enumerate(item2[:-1]):
        try:
            bigram[chara][item2[i+1]] += 1
        except:
            bigram[chara][item2[i+1]] = 1

In [19]:
outerKeys = dict()

for item in bigram.keys():
    for stuff in bigram[item]:
        try:
            outerKeys[item] += bigram[item][stuff]
        except:
            outerKeys[item] = bigram[item][stuff]

            
for item in bigram.keys():
    try:
        del bigram[item]['$']
    except:
        pass
    
    for stuff in bigram[item]:
        bigram[item][stuff] = (1.0*bigram[item][stuff])/outerKeys[item]


def bigramM(mask,guessed):
    currMax = 0.0
    
    currBi = '_'
    if len(mask) == mask.count('_'):
        return unig(mask,guessed).split('_')[0]
    else:
        for i,item in enumerate(mask):
            try:
                if item != '_' and mask[i+1] == '_':
                    sorted_x = sorted(bigram[item].items(), key=operator.itemgetter(1),reverse=True)
                    for thing in sorted_x:
                        if thing[1] > currMax and thing[0] not in guessed:
                            currMax = thing[1]
                            currBi = thing[0]
                            break
            except IndexError:
                pass

        return currBi

hangman('whatever', bigramM, 8, False)               

8

In [20]:
totMist = 0
corrs = 0
for i,item in enumerate(test):
    mist = hangman(item, bigramM, 8, False)
    totMist += mist
    if mist < 8:
        corrs += 1
print (corrs,totMist)

2471 82243


### N-gram Models

In [21]:
from collections import defaultdict

def ngramModel(train,ngt,countsOnly=1):
    ng = ngt - 1
    bigram = defaultdict(dict)
    outerKeys =dict()
    for item in train:
        item2 = '$'+item+'$'
        for i,chara in enumerate(item2):
            if i+ng >= len(item2)-1:
                break
            try:
                bigram[item2[i:i+ng]][item2[i+ng]] += 1
            except:
                try:
                    bigram[item2[i:i+ng]][item2[i+ng]] = 1
                except IndexError:
                    pass
    for item in bigram.keys():
        for stuff in bigram[item]:
            try:
                outerKeys[item] += bigram[item][stuff]
            except:
                outerKeys[item] = bigram[item][stuff]
    if countsOnly != 1:
        for item in bigram.keys():
            try:
                del bigram[item]['$']
            except:
                pass
            for stuff in bigram[item]:
                bigram[item][stuff] = (1.0*bigram[item][stuff])/outerKeys[item]
    
    return bigram,outerKeys

In [22]:
ngModels = dict()
outKeys = dict()

for i in range(2,6):
    ngModels[i],outKeys[i] = ngramModel(train,i)

In [23]:
ngModels[5]

defaultdict(dict,
            {'mdbi': {'o': 1},
             'xodu': {'s': 3},
             'xodt': {'e': 1},
             'xode': {'n': 1, 'r': 2, 's': 1, 't': 1},
             'xodo': {'n': 1},
             'xodi': {'a': 1, 'o': 1, 'u': 1},
             'fsou': {'n': 1, 't': 2},
             '\xbeark': {'o': 1},
             '\xbeare': {'v': 1},
             'c\xc6\xb0\xe1': {'\xbb': 1},
             'kid\xc5': {'\x8d': 1},
             '\xc5\x9bok': {'a': 1},
             '\xa7at\xc4': {'\xb1': 1},
             '\xbears': {'k': 1},
             'jrpg': {'s': 1},
             'tcbu': {'l': 1},
             'tcbi': {'o': 1},
             'jhes': {'s': 1},
             'jher': {'e': 1, 'i': 2, 'o': 1},
             'jhet': {'t': 1},
             '$wde': {'f': 1, 'l': 1, 'q': 1, 't': 1, 'v': 1},
             'jhee': {'l': 3},
             'jhei': {'d': 1},
             'bbqi': {'n': 1},
             '$m\xc5\x8d': {'r': 1},
             'jhel': {'u': 1, 'y': 1},
             'jheo': {'a

## Backoff Models

- use  n-gram  if  you  have  good  evidence
- otherwise  (n-1)-gram, (n-2)-gram .... bigram, unigram .
- How to come up with good evidence

In [24]:
from __future__ import print_function 

def hangman2(secret_word, guesser, max_mistakes=8, verbose=True,param=20):
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed,param)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

In [25]:
def ngramM(mask,guessed,paramList):
    proList = list()
    maski = '$' +  ''.join(mask) + '$'
    if len(mask) == mask.count('_'):
        return unigPos(mask,guessed)
    maskSort = sorted([item[-4:] for item in maski.split('_') if len(item)>=1 and '$'!=item[-1]],key=lambda x: len(x),reverse=True)
    for item in maskSort:
        for i in range(4):
            proAdded = 0
            try:
                if outKeys[len(item[i:])+1][item[i:]] > paramList[i]:
                    pro = sorted(ngModels[len(item[i:])+1][item[i:]].items(),key=operator.itemgetter(1),reverse=True)
                    for pros in pro:
                        if pros[0] not in guessed:
                            proList.append((pros[0],pros[1],pros[1]*1.0/outKeys[len(item[i:])+1][item[i:]],len(item[i:])+1))
                            proAdded = 1
                            break
                    if proAdded == 1:
                        break
            except (IndexError,KeyError) as e:
                pass
                
    #print(proList)
    if len(proList) == 0:
        return unigPos(mask,guessed)
    return sorted(proList, key=lambda vertex: (vertex[3], vertex[2]),reverse=True)[0][0]

In [ ]:
maxparai = 0
maxCorr = 0


for uni in  range(1,20):
    totMist = 0
    corrs = 0
    parai = [uni,uni,uni,uni]
    for i,item in enumerate(test):
        mist = hangman2(item, ngramM, 8, False,parai)
        totMist += mist
        if mist < 8:
            corrs += 1
    if corrs > maxCorr:
        print ('Best - ',parai,corrs,totMist)
        maxparai = parai
        maxCorr = corrs


Best -  [1, 1, 1, 1] 4514 76100
Best -  [2, 2, 2, 2] 4515 76099
Best -  [5, 5, 5, 5] 4516 76086


In [ ]:
maxparai = 0
maxCorr = 0


for uni in  range(1,20):
    for bi in range(1,20):
        for tri in range(1,20):
            for quad in range(1,20):
                totMist = 0
                corrs = 0
                parai = [quad,tri,bi,uni]
                for i,item in enumerate(test):
                    mist = hangman2(item, ngramM, 8, False,parai)
                    totMist += mist
                    if mist < 8:
                        corrs += 1
                if corrs > maxCorr:
                    print ('Best - ',parai,corrs,totMist)
                    maxparai = parai
                    maxCorr = corrs

## Interploation
- mix  unigram,  bigram,  trigram and so on

$$ P(w_n|w_{n-2},w_{n-1}) = \lambda_1 P(w_n|w_{n-2},w_{n-1}) + \lambda_2 P(w_n|w_{n-1}) + \lambda_3 P(w_n)   $$

In [ ]:
def ngraInterp(mask,guessed,paramList=[0.1,0.2,0.3,0.4]):
    proList = list()
    maski = '$' +  ''.join(mask) + '$'
    if len(mask) == mask.count('_'):
        return unigPos(mask,guessed)
    maskSort = sorted([item[-4:] for item in maski.split('_') if len(item)>=1 and '$'!=item[-1]],key=lambda x: len(x),reverse=True)
    #print (maskSort)
    for item in maskSort:
        candidates = dict()
        for i in range(-1*(len(item)-1),1):
            #print(item,item[-i:],-i,len(item[-i:]))
            proAdded = 0
            try:
                pro = sorted(ngModels[len(item[-i:])+1][item[-i:]].items(),key=operator.itemgetter(1),reverse=True)
                for pros in pro:
                    if pros[0] not in guessed:
                        try:
                            candidates[pros[0]] += paramList[-i]* pros[1]*1.0/outKeys[len(item[-i:])+1][item[-i:]]
                        except:
                            candidates[pros[0]] = paramList[-i]* pros[1]*1.0/outKeys[len(item[-i:])+1][item[-i:]]
            except (IndexError,KeyError) as e:
                pass
        finMark = list(sorted(candidates.items(), key=operator.itemgetter(1),reverse=True)[0])
        finMark.append(item)
        proList.append(tuple(finMark))
                
    #print(proList)
    if len(proList) == 0:
        return unigPos(mask,guessed)
    #print(sorted(proList, key=lambda x: x[0],reverse=True)[0][0])
    return sorted(proList, key=lambda x: x[0],reverse=True)[0][0]

In [ ]:
hangman2('whatever', ngraInterp, 8, False,[0.9,0.3,1.3,0.4])             
totMist = 0
corrs = 0
for i,item in enumerate(test):
    mist = hangman2(item, ngraInterp, 8, False,[0.3,0.5,0.7,0.9])
    totMist += mist
    if mist < 8:
        corrs += 1
print (corrs,totMist)

In [ ]:
def ngraInterp(mask,guessed,paramList=[0.25,0.25,0.25,0.25]):
    proList = list()
    maski = '$' +  ''.join(mask) + '$'
    if len(mask) == mask.count('_'):
        return unig(mask,guessed)
    maskSort = sorted([item[-4:] for item in maski.split('_') if len(item)>=1 and '$'!=item[-1]],key=lambda x: len(x),reverse=True)
    #print (maskSort)
    for item in maskSort:
        candidates = dict()
        for i in range(-1*(len(item)-1),1):
            #print(item,item[-i:],-i,len(item[-i:]))
            proAdded = 0
            try:
                pro = sorted(ngModels[len(item[-i:])+1][item[-i:]].items(),key=operator.itemgetter(1),reverse=True)
                for pros in pro:
                    if pros[0] not in guessed:
                        try:
                            candidates[pros[0]] += paramList[-i]* pros[1]*1.0/outKeys[len(item[-i:])+1][item[-i:]]
                        except:
                            candidates[pros[0]] = paramList[-i]* pros[1]*1.0/outKeys[len(item[-i:])+1][item[-i:]]
            except (IndexError,KeyError) as e:
                pass
        finMark = list(sorted(candidates.items(), key=operator.itemgetter(1),reverse=True)[0])
        finMark.append(item)
        proList.append(tuple(finMark))
                
    #print(proList)
    if len(proList) == 0:
        return unigPos(mask,guessed)
    return sorted(proList, key=lambda x: x[0],reverse=True)[0][0]

In [ ]:
hangman2('whatever', ngraInterp, 8, False, [0.25,0.25,0.25,0.25])   

In [ ]:
totMist = 0
corrs = 0
for i,item in enumerate(test):
    mist = hangman2(item, ngraInterp, 8, False, [ 0.25,0.25,0.25,0.25])
    #[ 6.18284023  0.66143727  0.91415198  2.9306201 ]
    totMist += mist
    if mist < 8:
        corrs += 1
print (corrs,totMist)

## parameter Estimation

In [ ]:
def ngraInterpWeights(mask,guessed,paramList=[ 0.25,0.25,0.25,0.25]):
    proList = list()
    scores = defaultdict(dict)
    maski = '$' +  ''.join(mask) + '$'
    if len(mask) == mask.count('_'):
        return unig(mask,guessed),'nil'
    maskSort = sorted([item[-4:] for item in maski.split('_') if len(item)>=1 and '$'!=item[-1]],key=lambda x: len(x),reverse=True)
    #print (maskSort)
    for item in maskSort:
        candidates = dict()

        
        for i in range(-1*(len(item)-1),1):
            #print(item,item[-i:],-i,len(item[-i:]))
            proAdded = 0
            try:
                pro = sorted(ngModels[len(item[-i:])+1][item[-i:]].items(),key=operator.itemgetter(1),reverse=True)
                for pros in pro:
                    if pros[0] not in guessed:
                        try:
                            candidates[pros[0]] += paramList[-i]* pros[1]*1.0/outKeys[len(item[-i:])+1][item[-i:]]
                        except:
                            candidates[pros[0]] = paramList[-i]* pros[1]*1.0/outKeys[len(item[-i:])+1][item[-i:]]
                        scores[item+'_'+pros[0]][len(item[-i:])+1] =  paramList[-i]* pros[1]*1.0/outKeys[len(item[-i:])+1][item[-i:]]

            except (IndexError,KeyError) as e:
                pass
        finMark = list(sorted(candidates.items(), key=operator.itemgetter(1),reverse=True)[0])
        finMark.append(item)
        proList.append(tuple(finMark))
                
    #print(proList)
    if len(proList) == 0:
        return unig(mask,guessed),'nil'
    
    finVal = sorted(proList, key=lambda x: x[0],reverse=True)[0][0]
    newScores = defaultdict(dict)
    for item in scores.keys():
        if item.split('_')[1]== finVal:
            newScores[item] = scores[item]
    return finVal,newScores


def hangman3(secret_word, guesser, max_mistakes=8, verbose=True,param=20):
    tuples = open('filePar.csv','a')
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess,scores = guesser(mask, guessed,param)
        finScores = defaultdict(dict)
        for i in range(1,6):
            finScores[guess][i] = 0.0
        if scores != 'nil': 
            for item in scores.keys():
                for stuff in scores[item].keys():
                    try:
                        finScores[guess][stuff] += scores[item][stuff]
                    except KeyError:
                        print(item,stuff)

                
        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
                finScores[guess]['correct'] = 1
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                finScores[guess]['correct'] = 0
        if scores != 'nil':
            print(guess,finScores[guess][2],finScores[guess][3],finScores[guess][4],finScores[guess][5],finScores[guess]['correct'],file=tuples,sep=',')
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    tuples.close()
    return mistakes

In [ ]:
corrs = 0
totMist = 0 
tuples = open('filePar.csv','w')
tuples.close()
for i,item in enumerate(test):
    mist = hangman3(item, ngraInterpWeights, 8, False,[ 0.25,0.25,0.25,0.25])
    totMist += mist
    if mist < 8:
        corrs += 1
print (corrs,totMist)


In [ ]:
import pandas
params = pandas.read_csv('filePar.csv',names=['char','bi','tri','quad','five','label'])
params.groupby('label').count()

In [ ]:
from sklearn import linear_model
log_model = linear_model.LogisticRegression(max_iter=1000)
train_features = params[['bi','tri','quad','five']]
test_features = params['label']
log_model.fit(X = train_features ,
              y = test_features)

print(log_model.coef_,log_model.get_params())

In [ ]:
a = log_model.coef_[0]
b = [item/sum(a) for item in a]
b,b[::-1]


In [ ]:
corrs = 0
totMist = 0 
for i,item in enumerate(test):
    mist = hangman3(item, ngraInterpWeights, 8, False,b)
    totMist += mist
    if mist < 8:
        corrs += 1
print (corrs,totMist)


## Reference
 - [Stanford CS124 slides](https://web.stanford.edu/class/cs124/lec/languagemodeling.pdf)